In [1]:
# Importieren der benötigten Bibliotheken
import pandas as pd
import requests

In [2]:
# Von Opendata VVS die Haltestellen abrufen
df_vss = pd.read_csv("../../01_data/01_raw/vvs_haltestelle_j24.csv", encoding='ISO-8859-1', sep=";")

In [3]:
# Auf die S-Bahn Haltestellen beschränken
df_vss_s_bahn = df_vss[df_vss['Verkehrsmittel'].str.contains('S-Bahn')]
df_vss_s_bahn = df_vss_s_bahn.rename(columns={'#Name': 'name'})

In [4]:
name_mapping = {
    'Nürnberger Straße': 'Stuttgart Nürnberger Straße',
    'Sommerrain': 'Stuttgart-Sommerrain',
    'Malmsheim': 'Malmsheim',
    'Rutesheim': 'Rutesheim',
    'Weil der Stadt': 'Weil der Stadt',
    'Bietigheim': 'Bietigheim-Bissingen',
    'Kornwestheim': 'Kornwestheim Pbf',
    'Neuwirtsh. (Porschep.)': 'Neuwirtshaus (Porscheplatz)',
    'Benningen (N)': 'Benningen (Neckar)',
    'Freiberg (N)': 'Freiberg (Neckar)',
    'Nellmersbach': 'Nellmersbach',
    'Neustadt-Hohenacker': 'Neustadt-Hohenacker',
    'Schwaikheim': 'Schwaikheim',
    'Stetten-Beinstein': 'Stetten-Beinstein',
    'Geradstetten': 'Geradstetten',
    'Grunbach': 'Grunbach',
    'Weiler (R)': 'Weiler (Rems)',
    'Winterbach': 'Winterbach (b Schorndorf)',
    'Altbach': 'Altbach',
    'Mettingen': 'Esslingen-Mettingen',
    'Oberesslingen': 'Oberesslingen',
    'Zell': 'Esslingen-Zell',
    'Filderstadt': 'Filderstadt',
    'Flughafen/Messe': 'Stuttgart Flughafen/Messe',
    'Oberaichen': 'Oberaichen',
    'Weilimdorf Bf': 'Stuttgart-Weilimdorf',
    'Süd': 'Renningen Süd',
    'Magstadt': 'Magstadt',
    'Nord': 'Maichingen Nord',
    'Maichingen': 'Maichingen',
    'Sindelfingen': 'Sindelfingen',
    'Goldberg': 'Goldberg',
    'Erdmannhausen': 'Erdmannhausen',
    'Beutelsbach': 'Beutelsbach',
    'Kirchheim (T)': 'Kirchheim (Teck)',
    'Wernau (N)': 'Wernau (Neckar)',
    'Wendlingen (N)': 'Wendlingen (Neckar)',
    'Ötlingen': 'Kirchheim (Teck)-Ötlingen',
    'Herrenberg': 'Herrenberg',
    'Böblingen': 'Böblingen',
    'Höfingen': 'Höfingen',
    'Leinfelden': 'Leinfelden',
    'Ehningen': 'Ehningen (b Böblingen)',
    'Gärtringen': 'Gärtringen',
    'Nufringen': 'Nufringen',
    'Rohr': 'Stuttgart-Rohr',
    'Vaihingen': 'Stuttgart-Vaihingen',
    'Universität': 'Stuttgart Universität',
    'Österfeld': 'Stuttgart Österfeld',
    'Schwabstraße': 'Stuttgart Schwabstraße',
    'Stadtmitte': 'Stuttgart Stadtmitte',
    'Neckarpark': 'Stuttgart Neckarpark',
    'Untertürkheim': 'Stuttgart-Untertürkheim Pbf',
    'Obertürkheim': 'Stuttgart-Obertürkheim',
    'Hauptbahnhof (oben)': 'Stuttgart Hbf',
    'Hauptbahnhof (tief)': 'Stuttgart Hbf (tief)',
    'Feuerbach': 'Stuttgart-Feuerbach',
    'Feuersee': 'Stuttgart Feuersee',
    'Nordbahnhof': 'Stuttgart Nord',
    'Bad Cannstatt': 'Stuttgart-Bad Cannstatt',
    'Zuffenhausen': 'Stuttgart-Zuffenhausen',
    'Fellbach': 'Fellbach',
    'Ditzingen': 'Ditzingen',
    'Echterdingen': 'Echterdingen',
    'Hulb': 'Hulb',
    'Leonberg': 'Leonberg',
    'Renningen': 'Renningen',
    'Asperg': 'Asperg',
    'Ludwigsburg': 'Ludwigsburg',
    'Favoritepark': 'Favoritepark',
    'Tamm': 'Tamm (Württ)',
    'Burgstall (M)': 'Burgstall (Murr)',
    'Kirchberg (M)': 'Kirchberg (Murr)',
    'Marbach (N)': 'Marbach (Neckar)',
    'Backnang': 'Backnang',
    'Maubach': 'Maubach',
    'Korntal': 'Korntal',
    'Waiblingen': 'Waiblingen',
    'Winnenden': 'Winnenden',
    'Rommelshausen': 'Rommelshausen',
    'Schorndorf': 'Schorndorf',
    'Endersbach': 'Endersbach',
    'Esslingen (N)': 'Esslingen (Neckar)',
    'Plochingen': 'Plochingen'
}

df_vss_s_bahn['name'] = df_vss_s_bahn['name'].map(name_mapping)

df_vss_s_bahn.to_csv("../../01_data/02_intermediate/vvs_s_bahn.csv", index=False)

In [5]:
# Station Daten von der DB API abrufen
url = "https://apis.deutschebahn.com/db-api-marketplace/apis/station-data/v2/stations"

headers = {
    "DB-Client-ID": "8818fc1cad6ea1a87f4d399146f6488d",
    "DB-Api-Key": "12ecba5ccbd86f53b8de3a3818d98021",
    "accept": "application/json"
}

response = requests.get(url, headers=headers)

data = response.json()
data = data["result"]
stations_list = []

In [6]:
# Daten aus der API aus dem JSON-Format auslesen und in eine Liste speichern
for i in range(len(data)):
    try:
        name =  data[i]["name"]
        city = data[i]["mailingAddress"]["city"]
        zipcode = data[i]["mailingAddress"]["zipcode"]
        street = data[i]["mailingAddress"]["street"]
        state =  data[i]["federalState"]
        category = data[i]["category"]
        parking = data[i]["hasParking"]
        bicycle = data[i]["hasBicycleParking"]
        taxis = data[i]["hasTaxiRank"]
        stepless = data[i]["hasSteplessAccess"]
        car_rental = data[i]["hasCarRental"]
        evaNumbers = data[i]["evaNumbers"][0]["number"]
        long = data[i]["evaNumbers"][0]["geographicCoordinates"]["coordinates"][0]
        lat = data[i]["evaNumbers"][0]["geographicCoordinates"]["coordinates"][1]
    except:
        continue
    
    item = {
        "eva_nr" : evaNumbers,
        "name" : name, 
        "category": category, 
        "state" : state, 
        "city" : city, 
        "zipcode" : zipcode, 
        "street" : street,
        "parking" : parking,
        "bicycle" : bicycle,
        "taxis" : taxis,
        "stepless" : stepless,
        "car_rental" : car_rental, 
        "long" : long,
        "lat" : lat,
        }
    stations_list.append(item)

df_db_stations = pd.DataFrame(stations_list)

In [7]:
# Erst auf Baden-Württemberg beschränken
df_db_stations_bw = df_db_stations[df_db_stations["state"] == "Baden-Württemberg"]

In [8]:
# Nur nur die S-Bahn Haltestellen im VVS-Gebiet auswählen
s_bahn_stopps = ['Stuttgart Nürnberger Straße', 'Leinfelden', 'Höfingen', 'Stuttgart-Sommerrain', 'Malmsheim', 'Rutesheim', 'Weil der Stadt',
                 'Bietigheim-Bissingen', 'Kornwestheim Pbf', 'Neuwirtshaus (Porscheplatz)', 'Benningen (Neckar)', 'Freiberg (Neckar)',
                 'Nellmersbach', 'Neustadt-Hohenacker', 'Schwaikheim', 'Stetten-Beinstein', 'Geradstetten', 'Grunbach',
                 'Weiler (Rems)', 'Winterbach (b Schorndorf)', 'Altbach', 'Esslingen-Mettingen', 'Oberesslingen', 'Esslingen-Zell', 'Filderstadt', 'Stuttgart Flughafen/Messe',
                 'Oberaichen', 'Stuttgart-Weilimdorf', 'Renningen Süd', 'Magstadt', 'Maichingen Nord', 'Maichingen', 'Sindelfingen', 'Goldberg', 'Erdmannhausen',
                 'Beutelsbach', 'Kirchheim (Teck)', 'Wernau (Neckar)', 'Wendlingen (Neckar)', 'Kirchheim (Teck)-Ötlingen', 'Herrenberg', 'Ehningen (b Böblingen)',
                 'Gärtringen', 'Nufringen', 'Stuttgart-Rohr', 'Stuttgart-Vaihingen', 'Stuttgart Universität', 'Stuttgart Österfeld', 'Stuttgart Schwabstraße', 'Stuttgart Stadtmitte', 'Stuttgart Neckarpark',
                 'Stuttgart-Untertürkheim Pbf', 'Stuttgart-Obertürkheim', 'Hauptbahnhof (oben)', 'Hauptbahnhof (tief)', 'Stuttgart-Feuerbach', 'Stuttgart Feuersee', 'Stuttgart Nord',
                 'Stuttgart-Bad Cannstatt', 'Stuttgart-Zuffenhausen', 'Fellbach', 'Ditzingen', 'Echterdingen', 'Böblingen', 'Hulb', 'Leonberg', 'Renningen',
                 'Asperg', 'Ludwigsburg', 'Favoritepark', 'Tamm (Württ)', 'Burgstall (Murr)', 'Kirchberg (Murr)', 'Marbach (Neckar)', 'Backnang', 'Maubach', 'Korntal',
                 'Waiblingen', 'Winnenden', 'Rommelshausen', 'Schorndorf', 'Endersbach', 'Esslingen (Neckar)', 'Plochingen', 'Stuttgart Hbf']

df_db_stations_bw_vvs = df_db_stations_bw[df_db_stations_bw['name'].isin(s_bahn_stopps)]

new_item = {
    "eva_nr": 8098096,
    "name": "Stuttgart Hbf (tief)",
    "category": 1,
    "state": "Baden-Württemberg",
    "city": "Stuttgart",
    "zipcode": "70173",
    "street": "Arnulf-Klett-Platz 2",
    "parking": True,
    "bicycle": True,
    "taxis": True,
    "stepless": "yes",
    "car_rental": True,
    "long": 9.180228831,
    "lat": 48.78338649
}

# Hauptbahnhof (tief) hinzufügen (nicht in der DB API vorhanden) # Es müssen insgesamt 84 Stationen sein
df_db_stations_bw_vvs = pd.concat([df_db_stations_bw_vvs, pd.DataFrame(new_item, index=[0])], ignore_index=True)

In [9]:
# Spalten Formate anpassen
df_db_stations_bw_vvs['city'] = df_db_stations_bw_vvs['city'].replace('Leinfelden - Echterlingen', 'Leinfelden-Echterdingen')
df_db_stations_bw_vvs['city'] = df_db_stations_bw_vvs['city'].replace('Leinfelden - Echterdingen', 'Leinfelden-Echterdingen')
df_db_stations_bw_vvs['city'] = df_db_stations_bw_vvs['city'].replace('Stuttgart-Feuerbach', 'Feuerbach')
df_db_stations_bw_vvs['stepless'] = df_db_stations_bw_vvs['stepless'].replace('yes', 'Yes')
df_db_stations_bw_vvs['stepless'] = df_db_stations_bw_vvs['stepless'].replace('partial', 'Partial')

In [10]:
df_stationen_s_bahnnetz_stuttgart = pd.merge(df_db_stations_bw_vvs, df_vss_s_bahn, how='left', on="name", suffixes=('_db', '_vvs'))

In [11]:
df_stationen_s_bahnnetz_stuttgart = df_stationen_s_bahnnetz_stuttgart.drop(columns=['Name mit Ort', 'Nummer', 'Globale ID', 'GKZ', 'Linien (DIVA)', 'long', 'lat', 'Anzahl Linien'])

In [12]:
order = ['eva_nr', 'name', 'state', 'city', 'Gemeinde', 'Teilort', 'Landkreis', 'Tarifzonen', 'zipcode', 'street', 'category', 'parking', 'bicycle', 'taxis', 'stepless', 'car_rental', 'Verkehrsmittel', 'Linien (EFA)', 'Betriebszweige', 'X-Koordinate', 'Y-Koordinate']
df_stationen_s_bahnnetz_stuttgart = df_stationen_s_bahnnetz_stuttgart[order]

In [13]:
df_stationen_s_bahnnetz_stuttgart.rename(columns={'X-Koordinate': 'long', 'Y-Koordinate': 'lat'}, inplace=True)

In [14]:
df_stationen_s_bahnnetz_stuttgart['long'] = df_stationen_s_bahnnetz_stuttgart['long'].str.replace(',', '.').astype(float)
df_stationen_s_bahnnetz_stuttgart['lat'] = df_stationen_s_bahnnetz_stuttgart['lat'].str.replace(',', '.').astype(float)

In [15]:
df_stationen_s_bahnnetz_stuttgart['city'] = df_stationen_s_bahnnetz_stuttgart['city'].replace('Wernau', 'Wernau (Neckar)')
df_stationen_s_bahnnetz_stuttgart['city'] = df_stationen_s_bahnnetz_stuttgart['city'].replace('Tamm ', 'Tamm')
df_stationen_s_bahnnetz_stuttgart['city'] = df_stationen_s_bahnnetz_stuttgart['city'].replace('Winterbach ', 'Winterbach')
df_stationen_s_bahnnetz_stuttgart['city'] = df_stationen_s_bahnnetz_stuttgart['city'].replace('Ehningen ', 'Ehningen')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Benningen (N)', 'Benningen am Neckar')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Esslingen (N)', 'Esslingen am Neckar')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Freiberg (N)', 'Freiberg am Neckar')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Kirchberg (M)', 'Kirchberg an der Murr')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Kirchheim (T)', 'Kirchheim unter Teck')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Marbach (N)', 'Marbach am Neckar')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Wendlingen (N)', 'Wendlingen am Neckar')
df_stationen_s_bahnnetz_stuttgart['Teilort'] = df_stationen_s_bahnnetz_stuttgart['Teilort'].replace('Wernau (N)', 'Wernau (Neckar)')

In [16]:

df_stationen_s_bahnnetz_stuttgart['Tarifzonen'] = df_stationen_s_bahnnetz_stuttgart['Tarifzonen'].str.replace(',', ';').str.replace('"', '')
df_stationen_s_bahnnetz_stuttgart['Linien (EFA)'] = df_stationen_s_bahnnetz_stuttgart['Linien (EFA)'].str.replace(',', ';').str.replace('"', '')
df_stationen_s_bahnnetz_stuttgart['Betriebszweige'] = df_stationen_s_bahnnetz_stuttgart['Betriebszweige'].str.replace(',', ';').str.replace('"', '')

In [17]:
columns_order = ['eva_nr', 'name', 'state', 'city', 'Gemeinde', 'Teilort', 'Landkreis', 
                 'zipcode', 'street', 'long', 'lat', 'Verkehrsmittel', 'Linien (EFA)', 
                 'Betriebszweige', 'Tarifzonen', 'category', 'parking', 
                 'bicycle', 'taxis', 'stepless', 'car_rental']

# DataFrame in dieser Reihenfolge neu anordnen
df_stationen_s_bahnnetz_stuttgart = df_stationen_s_bahnnetz_stuttgart[columns_order]

In [18]:
df_stationen_s_bahnnetz_stuttgart.to_csv("../../01_data/01_raw/Stationen_S-Bahnnetz_Stuttgart.csv", index=False)